# Data Preparation

In [3]:
import pandas
import matplotlib.pyplot as plt
from collections import Counter
import string
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras import losses

wf = pandas.read_csv("./EMPOLITICON.csv")
wf

2025-05-23 17:46:29.014225: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 17:46:29.647557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-23 17:46:29.861289: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-23 17:46:29.937291: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-23 17:46:27.996746: I tensorflow/core/platform/cpu_feature_guar

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,16/07/2021,Vladimir Putin,Meeting of APEC Economic Leaders,"Madam Chair,\n\nColleagues,\n\nFirst of all, ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,OPTIMISM,DEVELOPMENT
1,Russia,2021-09-05 00:00:00,Vladimir Putin,Victory Parade on Red Square,"Citizens of Russia,\n\nDear veterans,\n\nComra...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,JOY,NATIONALISM
2,Russia,2021-08-04 00:00:00,Vladimir Putin,Meeting on the results of implementing Preside...,"Good afternoon, colleagues.\n\nLet’s start.\n\...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
3,Russia,21-11-2020,Vladimir Putin,G20 Summit,"Colleagues,\n\nThe scope of problems humanity ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
4,Russia,20-11-2020,Vladimir Putin,Address to participants in Nuremberg Lessons f...,"Colleagues, friends,\n\nFirst of all, I would ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,UPSET,EXTREMISM
...,...,...,...,...,...,...,...,...,...,...
2005,USA,05/02/2013,Barack Obama,\nRemarks by the President,"Good afternoon, everybody. \n\nI wanted to sa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,DEVELOPMENT
2006,USA,04/02/2013,Barack Obama,\nRemarks by the President on Preventing Gun V...,"Hello, everybody. Please have a seat. Have a...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,OTHERS
2007,USA,02/02/2013,Barack Obama,\nWeekly Address: A Balanced Approach to Growi...,"Hi, everybody. \n\nIn the coming weeks, we fa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,OPTIMISM,DEVELOPMENT
2008,USA,28/01/2013,Barack Obama,\nRemarks by the President Before Meeting with...,"Well, Vice President Biden and I just want to...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,OPTIMISM,OTHERS


In [4]:
# shuffing the data
wf = wf.sample(frac=1).reset_index(drop=True)
wf

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,2021-03-26 00:00:00,Joe Biden,Statement by President Biden on the Attack on ...,More Americans voted in the 2020 elections tha...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,UPSET,OTHERS
1,China,2019-12-16 00:00:00,H.E. Wang Yi,Adhering to Multilateralism and Jointly Creati...,"Mr. Chair,\nColleagues,\n\nI am delighted to j...",State Councilor and Foreign Minister of the Pe...,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,NEUTRAL,INTERNATIONAL AFFAIRS
2,USA,08/08/2015,Barack Obama,\nWeekly Address: Reaffirming Our Commitment t...,"Hi, everybody. The right to vote is one of t...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,JOY,OTHERS
3,USA,14/07/2015,Barack Obama,\nStatement by the President on Iran,"Today, after two years of negotiations, the Un...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,INTERNATIONAL AFFAIRS
4,Russia,14-04-2000,Vladimir Putin,Speech at a State Duma Meeting to Consider the...,I would now like to deal very briefly with rem...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
...,...,...,...,...,...,...,...,...,...,...
2005,Russia,2008-07-05 00:00:00,Dmitry Medvedev,Speech at Inauguration Ceremony as President o...,"Citizens of Russia, friends!\n\nI have just sw...",President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,OPTIMISM,OTHERS
2006,USA,22/08/2015,Barack Obama,\nWeekly Address: It’s Time for Congress To Pa...,"Hi, everybody. Seven years after the worst ec...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,DEVELOPMENT
2007,USA,16/11/2013,Barack Obama,\nWeekly Address: Taking Control of America’s ...,"Hi, everybody. On Thursday, I visited a stee...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,JOY,DEVELOPMENT
2008,China,2015-07-24 00:00:00,H.E. Wang Yi,Video Message by Foreign Minister Wang Yi At t...,"Dear Colleagues,\nLadies and Gentlemen,\n\nLet...",State Councilor and Foreign Minister of the Pe...,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,OPTIMISM,DEVELOPMENT


In [5]:
def count_unique_word(wf):
    count = Counter()
    for word in wf['Emotion']:
        count[word] += 1
    return count

counter = count_unique_word(wf)

In [6]:
print(counter.most_common())

[('NEUTRAL', 589), ('OPTIMISM', 583), ('JOY', 525), ('UPSET', 313)]


Setting labels for a multi-classification problem

In [7]:
#labeling the emotion column such that the most common emotion is 0, the second most common is 1 and so on
for i in range(len(wf)):
    if wf['Emotion'][i] == counter.most_common()[0][0]:
        wf['Emotion'][i] = 0
    elif wf['Emotion'][i] == counter.most_common()[1][0]:
        wf['Emotion'][i] = 1
    elif wf['Emotion'][i] == counter.most_common()[2][0]:
        wf['Emotion'][i] = 2
    elif wf['Emotion'][i] == counter.most_common()[3][0]:
        wf['Emotion'][i] = 3

#these labels will be used to train the model

/tmp/ipykernel_1133/1047653752.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Emotion'][i] = 3
/tmp/ipykernel_1133/1047653752.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values 

In [8]:
wf

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,2021-03-26 00:00:00,Joe Biden,Statement by President Biden on the Attack on ...,More Americans voted in the 2020 elections tha...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,3,OTHERS
1,China,2019-12-16 00:00:00,H.E. Wang Yi,Adhering to Multilateralism and Jointly Creati...,"Mr. Chair,\nColleagues,\n\nI am delighted to j...",State Councilor and Foreign Minister of the Pe...,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,0,INTERNATIONAL AFFAIRS
2,USA,08/08/2015,Barack Obama,\nWeekly Address: Reaffirming Our Commitment t...,"Hi, everybody. The right to vote is one of t...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,OTHERS
3,USA,14/07/2015,Barack Obama,\nStatement by the President on Iran,"Today, after two years of negotiations, the Un...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,INTERNATIONAL AFFAIRS
4,Russia,14-04-2000,Vladimir Putin,Speech at a State Duma Meeting to Consider the...,I would now like to deal very briefly with rem...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
...,...,...,...,...,...,...,...,...,...,...
2005,Russia,2008-07-05 00:00:00,Dmitry Medvedev,Speech at Inauguration Ceremony as President o...,"Citizens of Russia, friends!\n\nI have just sw...",President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,1,OTHERS
2006,USA,22/08/2015,Barack Obama,\nWeekly Address: It’s Time for Congress To Pa...,"Hi, everybody. Seven years after the worst ec...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,DEVELOPMENT
2007,USA,16/11/2013,Barack Obama,\nWeekly Address: Taking Control of America’s ...,"Hi, everybody. On Thursday, I visited a stee...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,DEVELOPMENT
2008,China,2015-07-24 00:00:00,H.E. Wang Yi,Video Message by Foreign Minister Wang Yi At t...,"Dear Colleagues,\nLadies and Gentlemen,\n\nLet...",State Councilor and Foreign Minister of the Pe...,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,1,DEVELOPMENT


In [9]:
# wf = wf.head(739)

In [10]:
wf.iloc[76]['Text_of_Speech']

'THE PRESIDENT:  Thank you very much.  Mr. Speaker, Mr. Vice President, members of Congress, the First Lady of the United States — (applause) — and citizens of America:\n\nTonight, as we mark the conclusion of our celebration of Black History Month, we are reminded of our nation’s path towards civil rights and the work that still remains to be done.  (Applause.)  Recent threats targeting Jewish community centers and vandalism of Jewish cemeteries, as well as last week’s shooting in Kansas City, remind us that while we may be a nation divided on policies, we are a country that stands united in condemning hate and evil in all of its very ugly forms.  (Applause.)\n\nEach American generation passes the torch of truth, liberty and justice in an unbroken chain all the way down to the present.  That torch is now in our hands.  And we will use it to light up the world.  I am here tonight to deliver a message of unity and strength, and it is a message deeply delivered from my heart.  A new chap

In [11]:
## one thing we can do is to use the model from the presidential classifier and then, if a given speech is classified
## as angry or some other negative mood, and it includes the word "US, NATO", or something related to the west, we classify it as adversarial

In [12]:
#removing the punctuation

## Yet to apply this to the data

def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

In [13]:
#removing the stop words (commonly used words that do not add much meaning to a sentence)

## Yet to actually apply this function to the data

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    #removes stop words
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)


[nltk_data] Downloading package stopwords to /home/yash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
#cleaning up the speeches so that common stopwords and puntuation are removed
for i in range(len(wf['Text_of_Speech'])):
    wf['Text_of_Speech'][i] = remove_punctuation(wf['Text_of_Speech'][i])
    wf['Text_of_Speech'][i] = remove_stopwords(wf['Text_of_Speech'][i])

/tmp/ipykernel_1133/2134825500.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Text_of_Speech'][i] = remove_punctuation(wf['Text_of_Speech'][i])
/tmp/ipykernel_1133/2134825500.py:4: FutureWarning: ChainedAssignmentError: behaviour wi

In [15]:
#drop rows without string in speech
for i in range(len(wf) - 1, -1, -1):
        if not (isinstance(wf.iloc[i]['Text_of_Speech'], str)):
                wf = wf.drop(i)

In [16]:
#counting the number of unique words in each speech


def count_unique_word(wf_training):
    count = Counter()
    for i in range(len(wf_training)):
        for word in wf_training.iloc[i]['Text_of_Speech'].split():
            count[word] += 1
    return count

counter = count_unique_word(wf)

In [17]:
counter

Counter({'people': 9977,
         '—': 8222,
         'new': 6587,
         'world': 6530,
         'work': 6325,
         'us': 6261,
         'china': 6246,
         'countries': 6156,
         'development': 6030,
         'one': 5367,
         'also': 5297,
         'country': 5190,
         'cooperation': 5139,
         'today': 4997,
         'need': 4856,
         'make': 4751,
         'thank': 4400,
         'years': 4319,
         'like': 4236,
         'must': 4210,
         'going': 4136,
         'want': 4067,
         'know': 4061,
         'would': 3998,
         'it’s': 3995,
         'time': 3944,
         'international': 3876,
         'economic': 3761,
         'many': 3709,
         'applause': 3698,
         'first': 3525,
         'we’re': 3520,
         '–': 3500,
         'year': 3467,
         'security': 3461,
         'united': 3394,
         'economy': 3278,
         'states': 3250,
         'that’s': 3248,
         'every': 3222,
         'great': 3150,
  

In [18]:
num_unique_words = len(counter)

In [19]:
print(len(wf))

2010


In [20]:
#80% training 20% testing split
wf_training = wf[:int(len(wf) * 0.8)]
wf_testing = wf[int(len(wf) * 0.8):]

In [21]:
wf_testing.reset_index(inplace=True)
wf_testing.drop('index', axis=1, inplace=True)
wf_testing

/tmp/ipykernel_1133/3894747268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_testing.drop('index', axis=1, inplace=True)


,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,02/04/2016,Barack Obama,\nWeekly Address: Securing the World from Nucl...,hi everybody week i’m speaking nuclear securit...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,INTERNATIONAL AFFAIRS
1,Russia,21-04-2006,Vladimir Putin,Opening Remarks at State Council Presidium Mee...,problem looking today global nature concerns r...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,3,DEVELOPMENT
2,China,2021-02-09 00:00:00,H.E. Xi Jinping,Keeping up the Momentum and Working Together f...,dear colleagues ladies gentlemen friends welco...,President,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,0,INTERNATIONAL AFFAIRS
3,USA,07/09/2013,Barack Obama,\nWeekly Address: Calling for Limited Military...,almost three weeks ago syria 1000 innocent peo...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,3,INTERNATIONAL AFFAIRS
4,Russia,2001-09-05 00:00:00,Vladimir Putin,Speech at the Parade Celebrating the 56th Anni...,greet veterans great patriotic war defended ri...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,2,NATIONALISM
...,...,...,...,...,...,...,...,...,...,...
397,Russia,2008-07-05 00:00:00,Dmitry Medvedev,Speech at Inauguration Ceremony as President o...,citizens russia friends sworn presidential oat...,President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,1,OTHERS
398,USA,22/08/2015,Barack Obama,\nWeekly Address: It’s Time for Congress To Pa...,hi everybody seven years worst economic crisis...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,DEVELOPMENT
399,USA,16/11/2013,Barack Obama,\nWeekly Address: Taking Control of America’s ...,hi everybody thursday visited steel plant clev...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,DEVELOPMENT
400,China,2015-07-24 00:00:00,H.E. Wang Yi,Video Message by Foreign Minister Wang Yi At t...,dear colleagues ladies gentlemen let begin ext...,State Councilor and Foreign Minister of the Pe...,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,1,DEVELOPMENT


In [22]:
wf_training.reset_index(inplace=True)
wf_training.drop('index', axis=1, inplace=True)
wf_training

/tmp/ipykernel_1133/1694703126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_training.drop('index', axis=1, inplace=True)


,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,2021-03-26 00:00:00,Joe Biden,Statement by President Biden on the Attack on ...,americans voted 2020 elections election nation...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,3,OTHERS
1,China,2019-12-16 00:00:00,H.E. Wang Yi,Adhering to Multilateralism and Jointly Creati...,mr chair colleagues delighted join 14th asem f...,State Councilor and Foreign Minister of the Pe...,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,0,INTERNATIONAL AFFAIRS
2,USA,08/08/2015,Barack Obama,\nWeekly Address: Reaffirming Our Commitment t...,hi everybody right vote one fundamental rights...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,OTHERS
3,USA,14/07/2015,Barack Obama,\nStatement by the President on Iran,today two years negotiations united states tog...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,INTERNATIONAL AFFAIRS
4,Russia,14-04-2000,Vladimir Putin,Speech at a State Duma Meeting to Consider the...,would like deal briefly remarks think cannot l...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
...,...,...,...,...,...,...,...,...,...,...
1603,USA,2020-09-17 00:00:00,Donald Trump,Remarks by President Trump at the White House ...,president thank much thank mike great vice pre...,President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,3,NATIONALISM
1604,United Kingdom,23 February 2012,David Cameron,Business in the Community speech,thank introduction thank welcome ladies gentle...,Prime Minister,David Cameron,https://www.gov.uk/government/speeches/busines...,0,OTHERS
1605,USA,2021-03-21 00:00:00,Joe Biden,Statement by President Biden on the Internatio...,one core values beliefs bring us together amer...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,0,EXTREMISM
1606,USA,2021-03-19 00:00:00,Joe Biden,Statement by President Biden on the COVID-⁠19 ...,jill share nation’s grief outrage horrific kil...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,3,EXTREMISM


In [23]:
wf_training

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,2021-03-26 00:00:00,Joe Biden,Statement by President Biden on the Attack on ...,americans voted 2020 elections election nation...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,3,OTHERS
1,China,2019-12-16 00:00:00,H.E. Wang Yi,Adhering to Multilateralism and Jointly Creati...,mr chair colleagues delighted join 14th asem f...,State Councilor and Foreign Minister of the Pe...,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,0,INTERNATIONAL AFFAIRS
2,USA,08/08/2015,Barack Obama,\nWeekly Address: Reaffirming Our Commitment t...,hi everybody right vote one fundamental rights...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,OTHERS
3,USA,14/07/2015,Barack Obama,\nStatement by the President on Iran,today two years negotiations united states tog...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,INTERNATIONAL AFFAIRS
4,Russia,14-04-2000,Vladimir Putin,Speech at a State Duma Meeting to Consider the...,would like deal briefly remarks think cannot l...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
...,...,...,...,...,...,...,...,...,...,...
1603,USA,2020-09-17 00:00:00,Donald Trump,Remarks by President Trump at the White House ...,president thank much thank mike great vice pre...,President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,3,NATIONALISM
1604,United Kingdom,23 February 2012,David Cameron,Business in the Community speech,thank introduction thank welcome ladies gentle...,Prime Minister,David Cameron,https://www.gov.uk/government/speeches/busines...,0,OTHERS
1605,USA,2021-03-21 00:00:00,Joe Biden,Statement by President Biden on the Internatio...,one core values beliefs bring us together amer...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,0,EXTREMISM
1606,USA,2021-03-19 00:00:00,Joe Biden,Statement by President Biden on the COVID-⁠19 ...,jill share nation’s grief outrage horrific kil...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,3,EXTREMISM


In [24]:
training_sentences = wf_training.Text_of_Speech.to_numpy()
#training labels
training_labels = wf_training.Emotion.to_numpy()

validation_sentences = wf_testing.Text_of_Speech.to_numpy()
#training labels
validation_labels = wf_testing.Emotion.to_numpy()

In [25]:
training_sentences.shape, validation_sentences.shape

((1608,), (402,))

In [26]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(training_sentences)

In [27]:
word_index = tokenizer.word_index

In [28]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

In [ ]:
#padding sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

#max number of words per sequence
max_length = 1000

training_padded = pad_sequences(training_sequences, maxlen=max_length, padding="post", truncating="post")
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding="post", truncating="post")
training_padded.shape, validation_padded.shape

((1608, 1000), (402, 1000))

In [30]:
#LSTM Model
import tensorflow.keras as keras


model = keras.models.Sequential()

#input layer
#embedding layer converts the words into vectors of fixed size
model.add(layers.Embedding(num_unique_words, 32))
#dropout rate drops out a certain percentage of input data so that the
#model learns more robust ways of learning and does not overfit
# model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(5, activation='softmax'))
# model.summary()

model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.summary()


I0000 00:00:1748036802.152484    1133 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748036802.654940    1133 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748036802.655098    1133 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748036802.663673    1133 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748036802.663746    1133 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
loss = 'sparse_categorical_crossentropy'
optim = keras.optimizers.Adam(learning_rate=0.001) #from 0.001
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [32]:
print(training_padded.dtype, training_padded.shape)

int32 (1608, 1000)


In [33]:

training_labels = np.array(training_labels).astype(np.int64)
validation_labels = np.array(validation_labels).astype(np.int64)

In [34]:
print(training_padded.dtype, training_padded.shape)
print(training_labels.dtype, training_labels.shape)

int32 (1608, 1000)
int64 (1608,)


In [35]:
print(training_padded)

[[  74 1936 1304 ...    0    0    0]
 [ 326 2975  222 ...    0    0    0]
 [1702  171   53 ...    0    0    0]
 ...
 [  10 1119  389 ...    0    0    0]
 [2089  272 1224 ...    0    0    0]
 [  56   14 7727 ...    0    0    0]]


In [36]:
##making sure that overrepresentation of a class does not affect the model
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(training_labels),
    y=training_labels
)
class_weights = dict(enumerate(class_weights))

In [37]:
#model.fit(training_padded, training_labels, epochs=20, validation_data=(validation_padded, validation_labels), class_weight=class_weights, batch_size=32, verbose=2)

# Changing to a Trinomial Classification Model Trained on Outside Data

In [90]:
wf = pandas.read_csv("./news_sentiment_analysiscsv.csv")
wf = wf.sample(frac=1).reset_index(drop=True)
wf

,Source,Author,Title,Description,URL,Published At,Sentiment,Type
0,theage,Julie Power,"Paris, LA and now Jenolan Caves: Funicular rai...",With the roads to Jenolan Caves closed for 18 ...,https://www.theage.com.au/national/nsw/paris-l...,2024-07-18T19:00:00+00:00,negative,Technology
1,Essentially Sports,Vatsal Shah,"Paris Olympics: Jessica Pegula, Iga Swiatek, a...","Come July, the sporting world is gearing up fo...",https://www.essentiallysports.com/wta-tennis-n...,2024-07-18T18:06:13+00:00,positive,Sports
2,dvidshub,Airman Monique Stober,From training to readiness: Air Force and VHA ...,Airmen assigned to the 6th Medical Group pose ...,https://www.dvidshub.net/image/8531506/trainin...,2024-07-16T16:51:41+00:00,negative,Health
3,myedmondsnews,MyEdmondsNews,Free cooking demonstration July 17 for cool bl...,The Verdant Health Commission will be hosting ...,https://myedmondsnews.com/2024/07/free-cooking...,2024-07-14T14:41:28+00:00,positive,Health
4,lulegacy,NaN,Strategic Buy Lights Up This Biotech Stock: Ti...,Life sciences company Illumina Inc. (NASDAQ: I...,https://www.marketbeat.com/stock-ideas/strateg...,2024-07-16T11:10:00+00:00,neutral,Science
...,...,...,...,...,...,...,...,...
3495,Lowell Sun,Tribune News Service,"TV Tinsel: NBC set for Olympic coverage, inclu...",NBC Sports is going all-out programming the 20...,https://www.lowellsun.com/2024/07/16/tv-tinsel...,2024-07-16T19:28:01+00:00,neutral,Sports
3496,etfdailynews,MarketBeat News,"NetScout Systems, Inc. (NASDAQ:NTCT) Shares Pu...",Empowered Funds LLC boosted its stake in NetSc...,https://www.etfdailynews.com/2024/07/14/netsco...,2024-07-14T11:46:45+00:00,positive,Technology
3497,dvidshub,Airman 1st Class Bryce Moore,STEM Fest [Image 4 of 9],A local elementary school student controls a r...,https://www.dvidshub.net/image/8528335/stem-fest,2024-07-14T16:22:09+00:00,positive,Technology
3498,Senego,Sidy Mohamet,Jeux Olympiques Paris 2024 : Le ministère des ...,"Le Ministère de la Jeunesse, des Sports et de ...",https://senego.com/jeux-olympiques-paris-2024-...,2024-07-12T20:32:45+00:00,positive,Sports


In [91]:
def count_unique_word(wf):
    count = Counter()
    for word in wf['Sentiment']:
        count[word] += 1
    return count

counter = count_unique_word(wf)

In [92]:
print(counter.most_common())

[('positive', 2134), ('neutral', 789), ('negative', 577)]


In [ ]:

for i in range(len(wf)):
    if wf['Sentiment'][i] == counter.most_common()[0][0]:
        wf['Sentiment'][i] = 0
    elif wf['Sentiment'][i] == counter.most_common()[1][0]:
        wf['Sentiment'][i] = 1
    elif wf['Sentiment'][i] == counter.most_common()[2][0]:
        wf['Sentiment'][i] = 2

/tmp/ipykernel_1133/3828753796.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Sentiment'][i] = 2
/tmp/ipykernel_1133/3828753796.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values

Same Process as the previous up to the LSTM Model

In [ ]:
# for i in range(len(wf['Description'])):
#     wf['Description'][i] = remove_punctuation(wf['Description'][i])
#     wf['Description'][i] = remove_stopwords(wf['Description'][i])

for i in range(len(wf) - 1, -1, -1):
        if not (isinstance(wf.iloc[i]['Description'], str)):
                wf = wf.drop(i)


counter = count_unique_word(wf)


num_unique_words = len(counter)



wf_training = wf[:int(len(wf) * 0.8)]
wf_testing = wf[int(len(wf) * 0.8):]



wf_testing.reset_index(inplace=True)
wf_testing.drop('index', axis=1, inplace=True)
wf_testing



wf_training.reset_index(inplace=True)
wf_training.drop('index', axis=1, inplace=True)
wf_training


training_sentences = wf_training.Description.to_numpy()
#training labels
training_labels = wf_training.Sentiment.to_numpy()
validation_sentences = wf_testing.Description.to_numpy()
#training labels
validation_labels = wf_testing.Sentiment.to_numpy()




tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(training_sentences)


word_index = tokenizer.word_index




training_sequences = tokenizer.texts_to_sequences(training_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)


max_length = 1000
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding="post", truncating="post")
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding="post", truncating="post")
training_padded.shape, validation_padded.shape


/tmp/ipykernel_1133/269933020.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Description'][i] = remove_punctuation(wf['Description'][i])
/tmp/ipykernel_1133/269933020.py:3: FutureWarning: ChainedAssignmentError: behaviour will chang

((2800, 1000), (700, 1000))

In [95]:
#LSTM Model



model = keras.models.Sequential()

#input layer
#embedding layer converts the words into vectors of fixed size
model.add(layers.Embedding(num_unique_words, 32))
#dropout rate drops out a certain percentage of input data so that the
model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [96]:
loss = 'sparse_categorical_crossentropy'
optim = keras.optimizers.Adam(learning_rate=0.001) #from 0.001
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [97]:
training_labels = np.array(training_labels).astype(np.int64)
validation_labels = np.array(validation_labels).astype(np.int64)

In [98]:

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(training_labels),
    y=training_labels
)
class_weights = dict(enumerate(class_weights))

In [99]:
print(class_weights)

{0: 0.5435837701417201, 1: 1.4675052410901468, 2: 2.087994034302759}


In [105]:
wf_training['Description'].iloc[3]

'verdant health commission hosting free cooking class wednesday july 17 show home cooks prepare chilled blueberry soup tangy colorful'

In [100]:
model.fit(training_padded, training_labels, epochs=30, validation_data=(validation_padded, validation_labels), class_weight=class_weights, batch_size=32, verbose=2)

Epoch 1/30
88/88 - 563s - 6s/step - accuracy: 0.4450 - loss: 1.0935 - val_accuracy: 0.4029 - val_loss: 1.0507
Epoch 2/30
88/88 - 554s - 6s/step - accuracy: 0.3982 - loss: 1.0394 - val_accuracy: 0.3886 - val_loss: 1.0100
Epoch 3/30
88/88 - 554s - 6s/step - accuracy: 0.4100 - loss: 1.0237 - val_accuracy: 0.3914 - val_loss: 1.0266
Epoch 4/30
88/88 - 556s - 6s/step - accuracy: 0.4114 - loss: 1.0211 - val_accuracy: 0.3914 - val_loss: 1.0505
Epoch 5/30
88/88 - 569s - 6s/step - accuracy: 0.4111 - loss: 1.0200 - val_accuracy: 0.4000 - val_loss: 0.9984
Epoch 6/30
88/88 - 547s - 6s/step - accuracy: 0.4164 - loss: 1.0173 - val_accuracy: 0.3929 - val_loss: 0.9877
Epoch 7/30


KeyboardInterrupt: 

# Using Other Outside Data

# Using the BERT Tokenizer